In [4]:
from sqlalchemy import create_engine
import pandas as pd

# Fill in your actual PostgreSQL credentials here
username = 'postgres'
password = 'krisdan29'
host = 'localhost'  # e.g., 'localhost'
port = '5432'       # default PostgreSQL port
dbname = 'pnta'

# Create the engine
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')

# Test the connection
with engine.connect() as conn:
    print("Connected:", not conn.closed)


Connected: True


Algorithm Nr. 1 - compute historical succsess of the venue and its events. 

In [5]:
venue = pd.read_sql("SELECT * FROM pnta.venue", con=engine)
event = pd.read_sql("SELECT * FROM pnta.event", con=engine)
user_event_checkin = pd.read_sql("SELECT * FROM pnta.user_event_checkin", con=engine)
user_event_like = pd.read_sql("SELECT * FROM pnta.user_event_like", con=engine)
user_event_rating = pd.read_sql("SELECT * FROM pnta.user_event_rating", con=engine)


In [6]:
# Merge event with venue to associate events with venue metadata
event_venue = pd.merge(event, venue, on="venue_id", how="left")

# Aggregate user_event_checkin: count of check-ins per event
checkin_counts = user_event_checkin.groupby("event_id").size().reset_index(name="checkin_count")

# Aggregate user_event_like: count of likes per event
like_counts = user_event_like.groupby("event_id").size().reset_index(name="like_count")

# Aggregate user_event_rating: average rating and count per event
rating_agg = user_event_rating.groupby("event_id").agg(
    rating_avg=("rating", "mean"),
    rating_count=("rating", "count")
).reset_index()

# Merge all aggregates into event_venue
event_merged = event_venue.merge(checkin_counts, on="event_id", how="left")
event_merged = event_merged.merge(like_counts, on="event_id", how="left")
event_merged = event_merged.merge(rating_agg, on="event_id", how="left")

# Fill NaNs in counts with 0
event_merged["checkin_count"] = event_merged["checkin_count"].fillna(0).astype(int)
event_merged["like_count"] = event_merged["like_count"].fillna(0).astype(int)
event_merged["rating_count"] = event_merged["rating_count"].fillna(0).astype(int)
event_merged["rating_avg"] = event_merged["rating_avg"].fillna(0)


# Show the first few rows in your local notebook
event_merged.head(10)



,event_id,venue_id,name_x,description_x,event_date,start_time,end_time,created_at,description_y,manager_id,...,picture,price_rating,rating,close_hours,icon,open_hours,checkin_count,like_count,rating_avg,rating_count
0,1,58,Sharable bifurcated algorithm,"Sharable bifurcated algorithm with music, vibe...",2025-03-04,19:00:00,21:00:00,2025-02-22,"Club-inspired venue located in Copenhagen, kno...",7,...,http://placeimg.com/640/480/any,HIGH,4.1,00:00,club.png,19:00,21,29,4.272727,33
1,2,72,User-centric even-keeled encryption,User-centric even-keeled encryption with music...,2025-03-14,21:00:00,22:00:00,2025-02-21,"Young-inspired venue located in Copenhagen, kn...",3,...,http://placeimg.com/640/480/any,MEDIUM,4.7,23:00,bar.png,17:00,34,44,2.857143,28
2,3,217,Face-to-face client-driven support,"Face-to-face client-driven support with music,...",2025-03-05,17:00:00,18:00:00,2025-02-24,"Tavern-inspired venue located in Odense, known...",12,...,http://placeimg.com/640/480/any,MEDIUM,3.9,23:00,bar.png,20:00,25,40,3.038462,26
3,4,120,Customer-focused systematic support,Customer-focused systematic support with music...,2025-05-04,21:00:00,22:00:00,2025-04-14,"Young-inspired venue located in Copenhagen, kn...",18,...,http://placeimg.com/640/480/any,HIGH,3.0,23:00,bar.png,19:00,28,38,3.648649,37
4,5,102,Quality-focused background parallelism,Quality-focused background parallelism with mu...,2025-05-23,21:00:00,01:00:00,2025-05-13,"Acoustic-inspired venue located in Copenhagen,...",10,...,http://placeimg.com/640/480/any,MEDIUM,4.1,00:00,bar.png,17:00,23,20,3.366667,30
5,6,230,Optimized 5thgeneration algorithm,"Optimized 5thgeneration algorithm with music, ...",2025-05-15,19:00:00,20:00:00,2025-04-18,"Open Air-inspired venue located in Odense, kno...",8,...,http://placeimg.com/640/480/any,LOW,4.0,23:00,bar.png,16:00,31,44,3.200000,35
6,7,82,Cross-group didactic emulation,"Cross-group didactic emulation with music, vib...",2025-05-29,20:00:00,23:00:00,2025-05-18,"None-inspired venue located in Copenhagen, kno...",12,...,http://placeimg.com/640/480/any,LOW,4.9,23:00,bar.png,20:00,25,44,3.441176,34
7,8,80,Profit-focused real-time algorithm,"Profit-focused real-time algorithm with music,...",2025-03-28,19:00:00,20:00:00,2025-03-23,"Expats-inspired venue located in Copenhagen, k...",13,...,http://placeimg.com/640/480/any,HIGH,4.3,23:00,bar.png,20:00,27,34,4.136364,22
8,9,195,Realigned dedicated structure,"Realigned dedicated structure with music, vibe...",2025-03-13,19:00:00,22:00:00,2025-02-19,"Concept-inspired venue located in Aarhus, know...",12,...,http://placeimg.com/640/480/any,LOW,3.2,23:00,bar.png,16:00,32,38,3.206897,29
9,10,136,Streamlined tangible moratorium,"Streamlined tangible moratorium with music, vi...",2025-03-06,20:00:00,21:00:00,2025-02-19,"Young-inspired venue located in Copenhagen, kn...",9,...,http://placeimg.com/640/480/any,LOW,3.2,02:00,bar.png,20:00,37,50,4.464286,28


In [7]:
event_cleaned = event_merged.drop(columns=['name_x', 'description_x', 'description_y', 'created_at', 'manager_id', 'name_y', 'picture', 'icon']) \
                             .sort_values(by="venue_id") \
                             .reset_index(drop=True)
event_cleaned.head(20)


,event_id,venue_id,event_date,start_time,end_time,price_rating,rating,close_hours,open_hours,checkin_count,like_count,rating_avg,rating_count
0,1995,1,2023-07-12,20:00:00,02:00:00,HIGH,4.7,02:00,19:00,27,54,3.107143,28
1,1528,1,2021-12-06,22:00:00,00:00:00,HIGH,4.7,02:00,19:00,27,54,4.233333,30
2,7791,1,2022-08-13,17:00:00,21:00:00,HIGH,4.7,02:00,19:00,21,42,4.200000,25
3,5408,1,2022-12-22,22:00:00,02:00:00,HIGH,4.7,02:00,19:00,14,27,4.424242,33
4,5698,1,2021-07-03,21:00:00,02:00:00,HIGH,4.7,02:00,19:00,19,38,3.111111,27
5,7658,1,2023-04-23,18:00:00,22:00:00,HIGH,4.7,02:00,19:00,20,40,3.862069,29
6,7711,1,2025-03-29,20:00:00,02:00:00,HIGH,4.7,02:00,19:00,18,35,4.521739,23
7,8429,1,2022-12-13,17:00:00,21:00:00,HIGH,4.7,02:00,19:00,15,29,3.791667,24
8,6907,1,2023-06-25,18:00:00,23:00:00,HIGH,4.7,02:00,19:00,24,47,3.733333,30
9,8801,1,2024-11-12,21:00:00,03:00:00,HIGH,4.7,02:00,19:00,21,42,2.611111,36


In [8]:
# Convert event_date to datetime if it's not already
event_cleaned["event_date"] = pd.to_datetime(event_cleaned["event_date"], errors="coerce")

# Sort by venue and event_date
event_cleaned = event_cleaned.sort_values(by=["venue_id", "event_date"]).reset_index(drop=True)

# Historical count of past events at this venue
event_cleaned["venue_event_count_before"] = event_cleaned.groupby("venue_id").cumcount()

# Historical average check-ins per venue before each event
event_cleaned["venue_avg_checkins_before"] = (
    event_cleaned.groupby("venue_id")["checkin_count"]
    .expanding()
    .mean()
    .shift(1)
    .reset_index(level=0, drop=True)
)

# Historical average likes per venue before each event
event_cleaned["venue_avg_likes_before"] = (
    event_cleaned.groupby("venue_id")["like_count"]
    .expanding()
    .mean()
    .shift(1)
    .reset_index(level=0, drop=True)
)

# Historical average rating per venue before each event
event_cleaned["venue_avg_rating_before"] = (
    event_cleaned.groupby("venue_id")["rating_avg"]
    .expanding()
    .mean()
    .shift(1)
    .reset_index(level=0, drop=True)
)

event_cleaned.head(20)


,event_id,venue_id,event_date,start_time,end_time,price_rating,rating,close_hours,open_hours,checkin_count,like_count,rating_avg,rating_count,venue_event_count_before,venue_avg_checkins_before,venue_avg_likes_before,venue_avg_rating_before
0,3424,1,2020-07-09,22:00:00,00:00:00,HIGH,4.7,02:00,19:00,22,44,4.303030,33,0,NaN,NaN,NaN
1,5698,1,2021-07-03,21:00:00,02:00:00,HIGH,4.7,02:00,19:00,19,38,3.111111,27,1,22.000000,44.000000,4.303030
2,1604,1,2021-11-18,19:00:00,00:00:00,HIGH,4.7,02:00,19:00,22,44,2.571429,28,2,20.500000,41.000000,3.707071
3,1528,1,2021-12-06,22:00:00,00:00:00,HIGH,4.7,02:00,19:00,27,54,4.233333,30,3,21.000000,42.000000,3.328523
4,9367,1,2022-07-22,17:00:00,23:00:00,HIGH,4.7,02:00,19:00,19,37,3.166667,42,4,22.500000,45.000000,3.554726
5,7791,1,2022-08-13,17:00:00,21:00:00,HIGH,4.7,02:00,19:00,21,42,4.200000,25,5,21.800000,43.400000,3.477114
6,8429,1,2022-12-13,17:00:00,21:00:00,HIGH,4.7,02:00,19:00,15,29,3.791667,24,6,21.666667,43.166667,3.597595
7,5408,1,2022-12-22,22:00:00,02:00:00,HIGH,4.7,02:00,19:00,14,27,4.424242,33,7,20.714286,41.142857,3.625320
8,5756,1,2023-01-15,17:00:00,20:00:00,HIGH,4.7,02:00,19:00,19,38,3.151515,33,8,19.875000,39.375000,3.725185
9,7658,1,2023-04-23,18:00:00,22:00:00,HIGH,4.7,02:00,19:00,20,40,3.862069,29,9,19.777778,39.222222,3.661444


In [9]:
# Group by venue_id and calculate aggregate stats
venue_success_summary = (
    event_cleaned.groupby("venue_id")
    .agg(
        total_events=("event_id", "count"),
        avg_checkins=("checkin_count", "mean"),
        avg_likes=("like_count", "mean"),
        avg_event_rating=("rating_avg", "mean"),
        median_event_rating=("rating_avg", "median")
        
    )
    .reset_index()
)

# Display the result
venue_success_summary.head(50)


,venue_id,total_events,avg_checkins,avg_likes,avg_event_rating,median_event_rating
0,1,24,20.166667,39.916667,3.511331,3.690196
1,2,31,19.741935,39.161290,3.526812,3.733333
2,3,31,20.903226,41.580645,3.375727,3.407407
3,4,38,19.315789,38.315789,3.554642,3.567145
4,5,39,20.487179,40.615385,3.426747,3.360000
5,6,39,18.102564,35.769231,3.169233,3.470588
6,7,33,19.484848,38.666667,3.431787,3.441176
7,8,38,18.763158,37.184211,3.060291,3.206957
8,9,33,17.090909,34.000000,3.300795,3.457143
9,10,33,18.939394,37.484848,3.507653,3.655172


In [10]:
# Assign quintiles based on avg_checkins
venue_success_summary["venue_popularity_tier"] = pd.qcut(
    venue_success_summary["avg_checkins"],
    q=5,
    labels=[1, 2, 3, 4, 5]
).astype(int)

venue_success_summary.head(20)

# Save the venue popularity summary to a CSV file
venue_success_summary.to_csv("venue_success_summary.csv", index=False)
